# `Setup`

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime as dt
from math import ceil
import gc

gc.collect()
try:
    # Mounting Colab Drive if possible
    from google.colab import drive
    drive.mount('/content/drive')

    # Cloning repo for colab
    if 'aml_itu' in os.getcwd():
        %cd aml_itu/
        !git pull https://github.com/RasKrebs/aml_itu
        !git checkout -b sparse_filtering
    else:
        !git clone -b sparse_test https://github.com/RasKrebs/aml_itu
        %cd aml_itu/
    os.environ["COLAB"] = "True"

except:
    # Changing directory into aml_itu
    if os.getcwd().split('/')[-1] != 'aml_itu': os.chdir(os.path.abspath('.').split('aml_itu/')[0]+'aml_itu')
    !git pull origin main --ff-only
    os.environ["COLAB"] = "False"

# Utils Import
from utils.helpers import *
from utils.StatefarmPytorchDataset import StateFarmDataset


# Torch
import torch
from torch import nn
import torchvision
torchvision.disable_beta_transforms_warning()
import torchvision.transforms as T
from torchvision.transforms import v2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# Install torchinfo, import if it's available
try:
  import torchinfo
except:
  !pip install torchinfo
  import torchinfo

from torchinfo import summary


# Printing current working directory
print(os.getcwd())

# Setting up device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print (f"GPU is available")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print('MPS device found.')
else:
    print ("No GPU available, using CPU instead")

From https://github.com/RasKrebs/aml_itu
 * branch            main       -> FETCH_HEAD
Already up to date.
/Users/alexanderries/aml_itu
MPS device found.


### `Config`

In [11]:
MODEL_NAME = 'SparseFilter_GrayScale'

# Loading the config file (if content is in workin directory must mean colab is being used)
config = load_config(eval(os.environ["COLAB"]))


# Training Images
train_img = config['dataset']['images']['train']

# Outputting config
config

{'dataset': {'name': 'state-farm-distracted-driver-detection',
  'colab_path': '/content/drive/MyDrive/aml-distracted-drivers-project',
  'data': '../state-farm-distracted-driver-detection/driver_imgs_list.csv',
  'images': {'train': '../state-farm-distracted-driver-detection/imgs/train',
   'test': '../state-farm-distracted-driver-detection/imgs/test'},
  'class_mapping': {'c0': 'safe driving',
   'c1': 'texting - right',
   'c2': 'talking on the phone - right',
   'c3': 'texting - left',
   'c4': 'talking on the phone - left',
   'c5': 'operating the radio',
   'c6': 'drinking',
   'c7': 'reaching behind',
   'c8': 'hair and makeup',
   'c9': 'talking to passenger'}},
 'outputs': {'path': './outputs'},
 'modeling_params': {'batch_size': 32, 'epochs': 100}}

In [12]:
def save_model(model, model_name, epoch):
    """Function for saving model"""
    # Model name, with path
    timestamp = dt.datetime.now().strftime('%Y%m%d_%H%M%S')
    file = f'{model_name}_{timestamp}_epoch_{epoch+1}'
    name = os.path.join(config['outputs']['path'], model_name, file)

    # Make directory if not exists
    if not os.path.exists(os.path.join(os.path.join(config['outputs']['path'], model_name))):
        os.makedirs(os.path.join(os.path.join(config['outputs']['path'], model_name)))

    # Save model
    torch.save(model.state_dict(), f'{name}.pt')

## Sparse Filtring

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch
import gc
gc.collect()
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
from tqdm import tqdm
import time

from torch.utils.data import Subset
import torch
from torch.utils.data import DataLoader
import random
from torch.utils.data import Subset
import torch
from torch.utils.data import DataLoader
import random

# Sparse Filter Class
class SparseFilter(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SparseFilter, self).__init__()
        self.weights = nn.Parameter(torch.randn(input_dim, output_dim) * 0.5)
        self.epsilon = 1e-8

    def soft_abs(self, value):
        return torch.sqrt(value ** 2 + self.epsilon)

    def forward(self, x):
        #print(x.shape)
        #print(self.weights.shape)
        first = torch.matmul(x, self.weights)
        second = self.soft_abs(first)
        third = second / torch.sqrt(torch.sum(second ** 2, axis=0) + self.epsilon)
        fourth = third / torch.sqrt(torch.sum(third ** 2, axis=1)[:, None] + self.epsilon)
        return torch.sum(fourth)

In [14]:
v3_transform = v2.Compose([v2.Compose([
      v2.ToPILImage(),
      v2.Resize((240, 320)),
      v2.Grayscale(num_output_channels=1),
      v2.ToTensor(),
      v2.Normalize(mean=[0.485], std=[0.229]),
      torch.flatten
      ])
    ])

# Load the dataset without transformations
train_data = StateFarmDataset(config,
                              transform=v3_transform,  # Transformations
                              split='train',
                              target_transform=None)

# Generate random indices for the subset
subset_size = 10000
indices = torch.randperm(len(train_data)).tolist()
subset_indices = indices[:subset_size]

# Create a subset
train_subset = Subset(train_data, subset_indices)

# Create a DataLoader for the subset
train_subset_loader = DataLoader(train_data, batch_size=32, num_workers=4, shuffle=True)

/Users/alexanderries/opt/anaconda3/lib/python3.9/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [15]:
def train_step(model, dataloader, optimizer, device, accumulation_steps=4):
    """Train step for a single epoch with gradient accumulation."""

    # Losses and accuracies
    train_loss, train_acc = 0, 0

    # Initialize the gradient
    optimizer.zero_grad()

    for i, data in enumerate(dataloader):

        # Extracting data and labels + moving to device
        imgs, labels = data
        imgs = imgs.to(device)

        # Forward pass
        loss = model(imgs)

        # Normalize the loss to account for accumulation
        normalized_loss = loss / accumulation_steps

        # Backward pass (accumulates gradients over multiple backward steps)
        normalized_loss.backward()

        # Step with optimizer every 'accumulation_steps' iterations
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(dataloader):
            optimizer.step()
            optimizer.zero_grad()

        # Update train loss
        train_loss += loss.item()

    # Return average train loss
    return train_loss / len(dataloader)

In [16]:
def visualize_training(history, num_epochs=50):

    # Generate Figure
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    # Loss Plots
    sns.lineplot(y=history['train_loss'], x=list(range(len(history['train_loss']))), ax=axs[0], label='Train Loss')
    sns.lineplot(y=history['val_loss'], x=list(range(len(history['val_loss']))), ax=axs[0], label='Validation Loss')
    axs[0].set_ylabel('Cross Entropy Loss')
    axs[0].set_xlabel('Epochs')
    axs[0].set_xlim(0, num_epochs)

    # Accuracy Plots
    sns.lineplot(y=history['train_acc'], x=list(range(len(history['train_acc']))), ax=axs[1], label='Train Accuracy')
    sns.lineplot(y=history['val_acc'], x=list(range(len(history['val_acc']))), ax=axs[1], label='Validation Accuracy')
    axs[1].set_ylabel('Accuracy')
    axs[1].set_xlabel('Epochs')
    axs[1].set_xlim(0, num_epochs)

    # Show plot
    plt.show()

In [17]:
def train_sparse_filter(model, train_dataloader, optimizer, epochs, device):
    """Model training method"""
    # History
    history = dict(train_loss=[],
                   train_acc=[],
                   val_loss=[],
                   val_acc=[])

    # Loop through epochs
    for epoch in range(epochs):
        print(f'\nEpoch {epoch+1} of {epochs} started...')

        # Set model to train mode and do pass over data
        model.train(True)
        train_loss = train_step(model, train_dataloader, optimizer, device)

        print(f"Epoch {epoch+1} of {epochs} - Train loss: {train_loss:.5f}")

        # Save model
        save_model(model, MODEL_NAME, epoch)

        # Save train and val loss/acc
        history['train_loss'].append(train_loss)

        # Visualize every 5th epoch
        if (epoch + 1) % 5 == 0:
            visualize_training(history, epochs)

    return history

In [18]:
# Define and train the sparse filter model
input_dim = 240*320  # Input dimension
output_dim = 48*64  # Desired output dimension
sparse_filter_model1 = SparseFilter(input_dim, output_dim).to(device)  # Move model to GPU
learning_rate = 0.01
optimizer = optim.Adam(sparse_filter_model1.parameters(), lr=learning_rate)
epochs = 50



results = train_sparse_filter(model=sparse_filter_model1,
                train_dataloader=train_subset_loader,
                optimizer=optimizer,
                epochs=epochs,
                device=device)


Epoch 1 of 50 started...
MPS device found.
MPS device found.
MPS device found.
MPS device found.


RuntimeError: MPS backend out of memory (MPS allocated: 31.71 GB, other allocations: 1.07 GB, max allowed: 36.27 GB). Tried to allocate 3.52 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
# Save the model weights
torch.save(sparse_filter_model1.state_dict(), './outputs/SparseFilterWeights/sparse_filter_model_try4_50epochs.pth')

In [ ]:

torch.save(sparse_filter_model1.state_dict(), './drive/MyDrive')